# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `https://hds5210-data.s3.amazonaws.com/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [5]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
all_hospitals = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')

In [6]:
all_hospitals

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,292,3.2,2.1,4.8,NaN,04/01/2015,03/31/2018
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,688,13,11.0,15.5,NaN,07/01/2015,06/30/2018
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,291,4.3,2.6,6.8,NaN,07/01/2015,06/30/2018
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,411,8.8,6.7,11.4,NaN,07/01/2015,06/30/2018
4,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_HF,Death rate for heart failure patients,No Different Than the National Rate,869,12.7,10.7,15.0,NaN,07/01/2015,06/30/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91390,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,TX,78660,TRAVIS,(512) 654-6100,PSI_4_SURG_COMP,Deaths among Patients with Serious Treatable C...,Not Available,Not Available,Not Available,Not Available,Not Available,5.0,07/01/2016,06/30/2018
91391,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,TX,78660,TRAVIS,(512) 654-6100,PSI_6_IAT_PTX,Collapsed lung due to medical treatment,Not Available,Not Available,Not Available,Not Available,Not Available,5.0,07/01/2016,06/30/2018
91392,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,TX,78660,TRAVIS,(512) 654-6100,PSI_8_POST_HIP,Broken hip from a fall after surgery,Not Available,Not Available,Not Available,Not Available,Not Available,5.0,07/01/2016,06/30/2018
91393,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER PFLUGERVILLE,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,TX,78660,TRAVIS,(512) 654-6100,PSI_90_SAFETY,Serious complications,Not Available,Not Available,Not Available,Not Available,Not Available,5.0,07/01/2016,06/30/2018


In [7]:
import datetime
def convert_date_time(row):
    return datetime.datetime.strptime(row, r"%m/%d/%Y")


def get_mo_summary(all_hospitals):
    # filter on the state where it should be missouri
    mo_hospitals = all_hospitals[all_hospitals['State'] == 'MO']

    #datetime coversion for this I have used the datetime strptime method
    mo_hospitals['Start Date'] = mo_hospitals['Start Date'].apply(convert_date_time)
    mo_hospitals['End Date'] = mo_hospitals['End Date'].apply(convert_date_time)

    # filter denominator column with only available data and convert it to the integer column
    mo_hospitals = mo_hospitals[mo_hospitals['Denominator'] != 'Not Available']
    mo_hospitals['Denominator'] = mo_hospitals['Denominator'].apply(int)

    # group by on hospital name and perfrom required aggregation
    mo_summary = mo_hospitals.groupby(by = ['Facility Name']).agg(
        start_date=pd.NamedAgg(column='Start Date', aggfunc='min'),
        end_date=pd.NamedAgg(column='End Date', aggfunc='max'),
        number=pd.NamedAgg(column='Denominator', aggfunc='sum')
    )

    return mo_summary

mo_summary = get_mo_summary(all_hospitals)

<ipython-input-7-570c5ba516b4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['Start Date'] = mo_hospitals['Start Date'].apply(convert_date_time)
<ipython-input-7-570c5ba516b4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['End Date'] = mo_hospitals['End Date'].apply(convert_date_time)


In [8]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

* My dataset will be consisting of three main source they are JSON file data, CSV File data and website tables, for reading this data we have Pandas support for all these three sources.
* Here are some links which I feel I coul use for the datasources
  * https://www.ahd.com/definitions/qual_collection_periods.php
  * https://hcup-us.ahrq.gov/reports/statbriefs/sb278-Conditions-Frequent-Readmissions-By-Payer-2018.jsp
  * https://data.cms.gov/provider-data/dataset/9n3s-kdb3
  * https://github.com/scpike/us-state-county-zip/blob/master/geo-data.csv


#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

* HTML
* CSV
* JSON

#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

* I will be working on the aspects of hospital readmission rates, particularly what kind of treatments are reciving more readmits in the hospitals.
* Focused towards identifying which particular state, city or may be hospital is recieving more readmits, breaking down the dieases section wise.
* The real outcome here is that if this analysis is completed, we can clearly identify that in particular state of USA, in particular city there are few hsopitals which are reciveing readmits for this particular disease.
* These suggestions could help other pharma companies to supply required medication to these areas, kind of like focused marketing.



---



## Submit your work via GitHub as normal
